# The use of epsilon to prevent singularities

In [ ]:
import sympy as sm

We have a function `f(x)`.

In the simplest case, we're trying to fix a singularity at `x=0`:  
`f(x).subs(x, 0) == sm.S.NaN`

We need to make an epsilon version `f(x, eps)` so the value is not NaN:  
`f(x, eps).subs(x, 0) != sm.S.NaN`

And also the derivative is not NaN:  
`f(x, eps).diff(x).subs(x, 0) != NaN`

For value continuity it would be nice to match the limit:  
`f(x, eps).subs(x, 0).limit(eps, 0) == f(x).limit(x, 0)`

For derivative continuity it would be nice to match the limit: 
`f(x, eps).diff(x).subs(x, 0).limit(eps, 0) == f(x).diff(x).limit(x, 0)`

In [ ]:
def is_epsilon_correct(func, singularity=0, limit_direction="+", display_func=display):
    # type: (T.Callable) -> bool
    """
    Check epsilon handling for a function that accepts a single value and an epsilon.

    For epsilon to be handled correctly, the function must:
        1) evaluate to a non-singularity at x=singularity given epsilon
        2) linear approximation of the original must match that taken with epsilon then substituted to zero
    """
    # Create symbols
    x = sm.Symbol('x', real=True)
    epsilon = sm.Symbol('epsilon', positive=True)
    EPS = 1e-8
    TOL = sm.sqrt(EPS)

    is_correct = True

    # Evaluate expression
    expr_eps = func(x, epsilon)
    expr_raw = expr_eps.subs(epsilon, 0)

    if display_func:
        display_func('Expressions (raw / eps):')
        display_func(expr_raw)
        display_func(expr_eps)

    # Sub in zero
    expr_eps_at_x_zero = expr_eps.subs(x, singularity)
    if expr_eps_at_x_zero == sm.S.NaN:
        display_func('[ERROR] Epsilon handling failed, expression at 0 is NaN.')
        is_correct = False     

    # Take constant approximation at singularity and check equivalence
    value_x0_raw = sm.simplify(expr_raw.limit(x, singularity, limit_direction))
    value_x0_eps = expr_eps.subs(x, singularity)
    value_x0_eps_sub2 = sm.simplify(value_x0_eps.limit(epsilon, 0))
    if value_x0_eps_sub2 != value_x0_raw:
        if display_func:
            display_func('[ERROR] Values at x={} not match (raw / eps / eps.limit):'.format(singularity))
            display_func(value_x0_raw)
            display_func(value_x0_eps)
            display_func(value_x0_eps_sub2)
        is_correct = False
    
    # NOTE(hayk): Perhaps it's useful to be less strict and plug in small numerical values for x and eps
    #     value_x0_eps_sub = value_x0_eps.subs(epsilon, EPS)
    #     if abs(value_x0_raw.evalf() - value_x0_eps_sub.evalf()) > TOL:]
    #     derivative_x0_eps_sub = derivative_x0_eps.subs(epsilon, EPS)
    #     if abs(derivative_x0_raw.evalf() - derivative_x0_eps_sub.evalf()) > TOL:

    # Take linear approximation at singularity and check equivalence
    derivative_x0_raw = sm.simplify(expr_raw.diff(x).limit(x, singularity, limit_direction))
    derivative_x0_eps = expr_eps.diff(x).subs(x, singularity)
    derivative_x0_eps_sub2 = sm.simplify(derivative_x0_eps.limit(epsilon, 0))
    if derivative_x0_eps_sub2 != derivative_x0_raw:
        if display_func:
            display_func('[ERROR] Derivatives at x={} not match (raw / eps / eps.limit):'.format(singularity))
            display_func(derivative_x0_raw)
            display_func(derivative_x0_eps)
            display_func(derivative_x0_eps_sub2)
        is_correct = False

    return is_correct

# Test sin(x) / x

For the whole section below, let's pretend x is positive so $x = -\epsilon$ is not a fear. We'll address that later.

In [ ]:
# Original function fails, singular at x=0
assert is_epsilon_correct(
    lambda x, eps: sm.sin(x) / x
) == False

In [ ]:
# Original broken attempt
assert is_epsilon_correct(
    lambda x, eps: sm.sin(x) / (eps + x)
) == False

In [ ]:
# Additive on top/bottom works
assert is_epsilon_correct(
    lambda x, eps: (eps + sm.sin(x)) / (eps + x)
) == True

In [ ]:
# Replacing all x with (x + eps) works
assert is_epsilon_correct(
    lambda x, eps: sm.sin(x + eps) / (x + eps)
) == True

# Test (1 - cos(x)) / x

In [ ]:
# Original fails, singular at 0
assert is_epsilon_correct(
    lambda x, eps: (1 - sm.cos(x)) / x
) == False

In [ ]:
# Value passes if we just replace the denominator, because this one is ~ x**2 / x unlike the above which is ~ x / x
assert is_epsilon_correct(
    lambda x, eps: (1 - sm.cos(x)) / (x + eps)
) == False

In [ ]:
# Derivative also passes if we replace both
assert is_epsilon_correct(
    lambda x, eps: (1 - sm.cos(x + eps)) / (x + eps)
) == True

# Test x / sqrt(x**2)

In [ ]:
# Original fails, singular at 0
assert is_epsilon_correct(
    lambda x, eps: x / sm.sqrt(x**2)
) == False

In [ ]:
# Broken fix #1
assert is_epsilon_correct(
    lambda x, eps: x / sm.sqrt(x**2 + eps**2)
) == False

In [ ]:
# Broken fix #2
assert is_epsilon_correct(
    lambda x, eps: (x + eps) / sm.sqrt(x**2 + eps**2)
) == False

In [ ]:
# Broken fix #3, ugh
assert is_epsilon_correct(
    lambda x, eps: (x + eps) / (eps + sm.sqrt(x**2 + eps**2))
) == False

In [ ]:
# Working if you again replace all x with x + eps
assert is_epsilon_correct(
    lambda x, eps: (x + eps) / sm.sqrt((x + eps)**2)
) == True

# Test acos(x) / sqrt(1 - x^2) at 1

In [ ]:
# Original fails, singular at 1
assert is_epsilon_correct(
    lambda x, eps: sm.acos(x) / sm.sqrt(1 - x**2),
    singularity=1,
) == False

In [ ]:
# Working if you replace all x with x + eps
assert is_epsilon_correct(
    lambda x, eps: sm.acos(x + eps) / sm.sqrt(1 - (x + eps)**2),
    singularity=1,
) == True

# Test atan2(0, x)

In [ ]:
# Original is singular
assert is_epsilon_correct(
    lambda x, eps: sm.atan2(0, x)
) == False

In [ ]:
# This works
assert is_epsilon_correct(
    lambda x, eps: sm.atan2(0, x + eps)
) == True

# Handling negative x

If we consider an example from above like $sin(x + \epsilon) / (x + \epsilon)$, this can easily be singular for a negative $x$, specifically where $x = -\epsilon$. If $x$ were always negative, we could do $sin(x - \epsilon) / (x - \epsilon)$.

So to handle both cases we can use the sign function as: $sin(x + sign(x) * \epsilon) / (x + sign(x) * \epsilon)$. This gives us the behavior that it always pushes $x$ away from zero because $sign(+) = 1$ and $sign(-) = -1$. However, $sign(0) = 0$ which breaks the original zero point. To resolve this we can make a "no zero" version that arbitrarily picks the positive direction when exactly at zero.

We can implement this cleverly with the help of a min function:

In [ ]:
def sign_no_zero(x):
    # type: (T.Scalar) -> T.Scalar
    """
    Returns -1 if x is negative, 1 if x is positive, and 1 if x is zero (given a positive epsilon).
    """
    return 2 * sm.Min(sm.sign(x), 0) + 1

In [ ]:
# Test for sin(x) / x
assert is_epsilon_correct(
    lambda x, eps: (sm.sin(x + eps * sign_no_zero(x))) / (x + eps * sign_no_zero(x))
) == True

In [ ]:
# Test for x / sqrt(x**2)
assert is_epsilon_correct(
    lambda x, eps: (x + eps) / sm.sqrt((x + eps)**2)
) == True

In [ ]:
# Test for atan2(0, x)
assert is_epsilon_correct(
    lambda x, eps: sm.atan2(0, x + eps * sign_no_zero(x))
) == True

# Generalization

So far it seems like for a function $f(x)$ that is singular at $x=0$, the expression $f(x + \epsilon * snc(x))$ for a small positive $\epsilon$ will be non-singular and retain the same linear approximiation as $f(x)$. $snc$ is `sign_no_zero`. So we can easily write a function that does this substitution:

In [ ]:
def add_epsilon_sign(expr, var, eps):
    return expr.subs(var, var + eps * sign_no_zero(var))

# Alternative using Max
def add_epsilon_max(expr, var, eps):
    return expr.subs(var, sign_no_zero(var) * sm.Max(eps, sm.Abs(var)))

In [ ]:
# Check known example
assert is_epsilon_correct(
    lambda x, eps: add_epsilon_sign(sm.sin(x) / x, x, eps)
) == True

In [ ]:
# With Max
assert is_epsilon_correct(
    lambda x, eps: add_epsilon_max(sm.sin(x) / x, x, eps)
) == True

In [ ]:
# Try some more complicated thing nobody wants to epsilon by hand
assert is_epsilon_correct(
    lambda x, eps: add_epsilon_sign((x + sm.sin(x)**2) / (x * (1 - 1/x)), x, eps)
) == True

Another common case is a function $f(x)$ that is singular at $|x| = 1$, where we expect $x$ to be between $-1$ and $1$.  In this case we can use a similar idea, using $f(x - \epsilon * snc(x))$.  Functions to do this would be:

In [ ]:
def add_epsilon_near_1_sign(expr, var, eps):
    return expr.subs(var, var - eps * sign_no_zero(var))

# Alternative using Max/Min
def add_epsilon_near_1_clamp(expr, var, eps):
    return expr.subs(var, sm.Max(-1 + eps, sm.Min(1 - eps, var)))

In [ ]:
# Check known example
assert is_epsilon_correct(
    lambda x, eps: add_epsilon_near_1_sign(sm.cos(x) / sm.sqrt(1 - x**2), x, eps),
    singularity=1,
    limit_direction="-",
) == True

So in a sense we could just not write epsilons by hand and automatically add them in. However, all of this is assuming the only singularity is at $x=0$. It's easy to construct an arbitrary singularity, like $1/(1-x)$ for $x=1$. It's also easy to construct singularities where this method works symbolically, but relies on values like $\epsilon^2$ which are too small in actual floating point implementations (see the `Pose2.to_tangent` section below).  And we could have a composite of multiple functions, so it's hard to have global visibility into what all your singular points are. Sometimes parameter have a fixed range, sometimes things are normalized, etc.

So the question is how well can we generalize this single variable example with a single known singularity to multiple variables and only locally known singularities like at the places we add a division, square root, or atan2 operatiion?

# Case Studies

## Pose2.from_tangent

`Pose2.from_tangent` before epsilon handling looks like:

```python
def pose2_from_tangent(v):
    theta = v[2]
    R = Rot2.from_tangent([theta])
    
    a = sm.sin(theta) / theta
    b = (1 - sm.cos(theta)) / theta
    
    t = geo.Vector2(a * v[0] - b * v[1], b * v[0] + a * v[1])
    return geo.Pose2(R, t)
```

This has singularities in both `a` and `b` that we'd like to fix.  The initial version used:

```python
a = sm.sin(theta) / (theta + epsilon)
b = (1 - sm.cos(theta)) / (theta + epsilon)
```

For `a`, this falls under the $sin(x) / x$ example above, and `b` falls under the $(1 - cos(x)) / x$ example above, so we can use the fixes from those examples, modified to handle negative $x$ as in the Generalization section.

## Pose2.to_tangent

`Pose2.to_tangent` before epsilon handling looks like:

```python
def pose2_to_tangent():
    theta = self.R.to_tangent()[0]
    halftheta = 0.5 * theta
    a = (halftheta * sm.sin(theta)) / (1 - sm.cos(theta))
    
    V_inv = Matrix[[a, halftheta], [-halftheta, a]])
    t_tangent = V_inv * self.t
    return [t_tangent[0], t_tangent[1], theta]
```

This has a singularity in `a` that looks like:

In [ ]:
assert is_epsilon_correct(
    lambda theta, eps: (0.5 * theta * sm.sin(theta)) / (1 - sm.cos(theta))
) == False

We might think this can be fixed with with the $x = x + \epsilon$ trick:

In [ ]:
assert is_epsilon_correct(
    lambda theta, eps: add_epsilon_sign((0.5 * theta * sm.sin(theta)) / (1 - sm.cos(theta)), theta, eps)
) == True

But in practice, the denominator's taylor series is $(1 - (1 - 0.5 \epsilon^2))$, and if $\epsilon$ is near machine precision, then the denominator will end up as exactly `1 - 1 == 0`.  This might be solved by adding $sqrt(\epsilon)$ instead, but that would introduce a large amount of error.  Instead, we do some algebra:

$$
\frac{0.5 \theta \sin(\theta)}{1 - \cos(\theta)}
= 
\frac{0.5 \theta \sin(\theta)}{1 - \cos(\theta)} \frac{1 + \cos(\theta)}{1 + \cos(\theta)}
= 
\frac{0.5 \theta \sin(\theta) (1 + \cos(\theta))}{1 - \cos^2(\theta)}
= 
\frac{0.5 \theta \sin(\theta) (1 + \cos(\theta))}{\sin^2(\theta)}
= 
\frac{0.5 \theta (1 + \cos(\theta))}{\sin(\theta)}
$$

Then, the only singularity is at $\sin(\theta) = 0$, which we can fix with:

In [ ]:
assert is_epsilon_correct(
    lambda theta, eps: (0.5 * (theta + eps) * (1 + sm.cos(theta))) / (sm.sin(theta) + eps)
) == True

## Rot3.to_tangent

`Rot3.to_tangent` before epsilon handling looks like:

```python
def logmap():
    norm = sm.sqrt(1 - self.q.w**2)
    tangent = 2 * self.q.xyz / norm * sm.acos(self.q.w)
    return tangent
```

Ignoring the `q.xyz` variables, the function has a singularity at `w == 1`, of the form
$$
\frac{\arccos(w)}{\sqrt{1 - w^2}}
$$

which can be fixed using either the `add_epsilon_near_1_sign` or `add_epsilon_near_1_clamp` methods described in the Generalization section.

# Multivariate functions

In [ ]:
x, y = sm.symbols('x, y', real=True)

In [ ]:
# Looking at a normalization function
epsilon = sm.Symbol('epsilon')
expr = x / sm.sqrt(x**2 + y**2)
sm.series(expr, x, n=2)
sm.simplify(expr.diff(x).limit(x, 0))

In [ ]:
# Simulate a normalization where y = 0
is_epsilon_correct(lambda x, eps: sm.Matrix([x + eps, 0]).normalized()[0])

In [ ]:
is_epsilon_correct(lambda x, eps: (x + eps) / sm.sqrt((x+eps)**2 + (0 + eps)**2))

In [ ]:
is_epsilon_correct(lambda x, eps: (x + eps) / sm.sqrt(eps**2 + (x + eps)**2))

In [ ]:
is_epsilon_correct(lambda x, eps: (x + eps) / sm.sqrt(eps**2 + (x + eps)**2))

# Sign simplification rules
Not directly related to epsilon, but why in tarnation don't these simplify? Seem like pretty simple rules to add.

In [ ]:
# Expect this to be x
sm.simplify(sm.sign(x) * sm.Abs(x))

In [ ]:
# Expect this to be |x|
sm.simplify(sm.sign(x) * x)

In [ ]:
# Expect this to be 1
sm.simplify(sm.sign(x) * sm.sign(x))

In [ ]:
# Expect this to be sign(x)
sm.simplify(x / sm.Abs(x))